<a href="https://colab.research.google.com/github/rchandran7/GoldfishLifespanPrediction/blob/main/GoldFishLifespanPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>